In [1]:
model = "gpt-35-turbo"
import os
openai_api_base = os.environ["OPENAI_API_BASE_URL"];

# 回顾

In [2]:
from openai import OpenAI
client = OpenAI(
    base_url = openai_api_base
)
def analyze_user_review(text):
    messages = []
    messages.append( {"role": "system", 
                      "content": """
                      You are an assistant. 
                      Please, analyze the user reviews according to the following instruction: 
                      If the review is postive, you should output 'Y', otherwise output 'N'
                      """})
    messages.append( {"role": "user", "content": text})
    response = client.chat.completions.create(
        model = model,
        messages=messages,
        temperature=0.5,
        max_tokens = 100
    )
    return response.choices[0].message.content

In [3]:
analyze_user_review("服务热情周到。")

'Y'

In [4]:
analyze_user_review("不理不睬。")

'N'

In [5]:
analyze_user_review("用餐环境差，等待时间长。")

'N'

In [6]:
!pip3 install --upgrade langchain

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


# 提示词模版

In [10]:
from langchain import PromptTemplate, OpenAI, LLMChain
from langchain.chat_models import ChatOpenAI 

prompt_template = "What is a good name for a company that makes {product}? And only return the best one."

llm = ChatOpenAI(model_name=model, temperature=0,openai_api_base= openai_api_base) #直接访问OpenAI的GPT服务
# llm = AzureChatOpenAI(deployment_name = deployment, model_name=model, temperature=0, max_tokens=200) # 通过Azure的OpenAI服务
llm_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate.from_template(prompt_template)
)
llm_chain.run("colorful socks")

'A great name for a company that makes colorful socks would be "VibrantHues".'

In [8]:
products = [{"product":"'cloudnative devops platform'"},
            {"product":"'Noise cancellation headphone'"}, 
            {"product":"colorful socks"}]
llm_chain.apply(products)

[{'text': 'The best name for a company that makes a cloud-native DevOps platform would be "CloudOps Innovations."'},
 {'text': 'The best name for a company that makes noise cancellation headphones could be "SilentSound."'},
 {'text': 'A fantastic name for a company that produces vibrant and colorful socks could be "ChromaSock."'}]

# API调用链
## HTTP request chain

In [11]:
from langchain import PromptTemplate, OpenAI, LLMChain
from langchain.chains import LLMRequestsChain
# from langchain.chat_models import AzureChatOpenAI
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name=model, temperature=0,openai_api_base= openai_api_base)
# llm = AzureChatOpenAI(deployment_name = deployment, model_name=model, temperature=0, max_tokens=200) #通过Azure的OpenAI服务

def query_baidu(question):
      template = """Between >>> and <<< are the raw search result text from web.
      Extract the answer to the question '{query}' or say "not found" if the information is not contained.
      Use the format
      Extracted:<answer or "not found">
      >>> {requests_result} <<<
      Extracted:"""

      PROMPT = PromptTemplate(
          input_variables=["query", "requests_result"],
          template=template,
      )

      inputs = {
          "query": question,
          "url": "http://www.baidu.com/s?wd=" + question.replace(" ", "+")
      }
      requests_chain = LLMRequestsChain(llm_chain = LLMChain(llm=llm, prompt=PROMPT), output_key="query_info", verbose=True)
      res = requests_chain.run(inputs)
      return res

In [12]:
query_baidu("今天北京天气？")



> Entering new LLMRequestsChain chain...

> Finished chain.


'Extracted: 今天多云转阴，最高气温9℃，最低气温2℃。西北风3-4级，阵风六七级。部分地区有雨夹雪或小雪。'

In [13]:
import os
# os.environ["SERPER_API_KEY"] = ""
# https://serper.dev
from langchain.utilities import GoogleSerperAPIWrapper
def query_web(question):
    search = GoogleSerperAPIWrapper()
    return search.run(question)


In [14]:
query_web("今天北京天气？")

'45°F'

# 调用链

In [25]:
from langchain import PromptTemplate, OpenAI, LLMChain
from langchain.chat_models import AzureChatOpenAI
from langchain.chains import SequentialChain
# from langchain.chat_models import AzureChatOpenAI
from langchain.chat_models import ChatOpenAI #直接访问OpenAI的GPT服务

llm = ChatOpenAI(model_name=model, temperature=0,openai_api_base= openai_api_base) #直接访问OpenAI的GPT服务
# llm = AzureChatOpenAI(deployment_name = deployment, model_name=model, temperature=0, max_tokens=200)

summarizing_prompt_template = """
Summarize the following content into a sentence less than 20 words:
---
{content}

"""
summarizing_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate.from_template(summarizing_prompt_template),
    output_key = "summary"
)

translating_prompt_template = """
translate "{summary}" into Chinese:

"""

translating_chain = LLMChain(
    llm = llm,
    prompt=PromptTemplate.from_template(translating_prompt_template),
    output_key = "translated"
)

overall_chain = SequentialChain(
    chains=[summarizing_chain, translating_chain],
    input_variables=["content"],
    output_variables=[ "summary","translated"],
    verbose=True
)


In [26]:
res = overall_chain("""
LangChain is a framework for developing applications powered by language models. It enables applications that are:

Data-aware: connect a language model to other sources of data
Agentic: allow a language model to interact with its environment
The main value props of LangChain are:

Components: abstractions for working with language models, along with a collection of implementations for each abstraction. Components are modular and easy-to-use, whether you are using the rest of the LangChain framework or not
Off-the-shelf chains: a structured assembly of components for accomplishing specific higher-level tasks
Off-the-shelf chains make it easy to get started. For more complex applications and nuanced use-cases, components make it easy to customize existing chains or build new ones.
""")

print("summary:"+res["summary"])

print("中文:"+res["translated"])



> Entering new SequentialChain chain...

> Finished chain.
summary:LangChain is a framework for creating language model-powered applications with data-aware and agentic capabilities, offering components and off-the-shelf chains for easy customization and implementation.
中文:LangChain是一个框架，用于创建具备数据感知和自主能力的语言模型驱动的应用程序，提供组件和现成链条，以便进行简单的定制和实施。
